In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-06-30"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
2048,2024-06-30,Nova Zelândia Nbl,00:00,Manawatu Jets,Canterbury Rams,12.00,1.01,184.5,1.80,1.86,0.0,0.00,0.00,nF7nGVRF,0.083333,0.990099,0.555556,0.537634,0.073432,571.932,359.527280,0.628619,0.0,0.000000,NaN,765.60,8.080,5.404924,0.668926,-130.0,141.948,74.554534,0.525224,3.0,0.000000,0.000000,109.14,1.412,0.648822,0.459506,94.0,66,107,11.60,1.02,408.944,162.012,1.194635,0.023184,NaN,-5.00,-1.000,-11.000000,0.000000,0.0,0.000000,3.16,0.632,0.015823,0.000000,0.0,0.000000
2049,2024-06-30,Nova Zelândia Nbl,03:30,Nelson Giants,Southland Sharks,1.08,7.00,173.5,1.81,1.85,0.0,0.00,0.00,GlCjFksM,0.925926,0.142857,0.552486,0.540541,0.068783,242.114,135.867108,0.561170,1.8,1.643168,0.912871,180.48,2.530,1.207622,0.477321,41.0,364.902,198.381195,0.543656,1.2,1.643168,1.369306,214.83,4.902,3.900240,0.795643,-28.0,96,93,1.88,2.31,205.040,554.540,1.036156,0.015456,NaN,-0.85,-0.170,-0.470588,0.000000,0.0,0.000000,-5.00,-1.000,-6.000000,0.000000,0.0,0.000000
2050,2024-06-30,República Dominicana Lnb,18:00,Titanes Del Licey,Caneros,1.43,2.68,170.5,1.85,1.85,0.0,0.00,0.00,29pyECEd,0.699301,0.373134,0.540541,0.540541,0.072435,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.430114,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
2051,2024-06-30,Venezuela Superliga,14:00,Gladiadores,Broncos,1.22,3.98,163.5,1.84,1.86,0.0,0.00,0.00,EZf6Sele,0.819672,0.251256,0.543478,0.537634,0.070928,100.074,21.156795,0.211412,2.4,1.341641,0.559017,87.74,1.156,0.102372,0.088557,22.0,148.632,29.201937,0.196471,1.2,1.643168,1.369306,132.88,1.636,0.202559,0.123813,-21.0,82,88,1.07,1.51,105.672,149.092,0.750621,0.007644,NaN,-0.09,-0.018,-12.222222,0.000000,0.0,0.000000,1.91,0.382,7.801047,0.000000,0.0,0.000000
2052,2024-06-30,Venezuela Superliga,14:00,Guaiqueries,Panteras,1.28,3.44,147.5,1.83,1.87,0.0,0.00,0.00,MabEQZI7,0.781250,0.290698,0.546448,0.534759,0.071948,166.582,53.093989,0.318726,3.0,0.000000,0.000000,246.48,2.094,0.654240,0.312436,43.0,210.058,75.126331,0.357646,1.2,1.643168,1.369306,263.53,2.880,0.868677,0.301624,-12.0,79,73,3.12,3.61,146.514,231.996,0.647182,0.015289,NaN,2.41,0.482,0.580913,0.000000,0.0,0.000000,1.92,0.384,6.354167,0.000000,0.0,0.000000
2053,2024-06-30,Venezuela Superliga,18:00,Centauros,Cocodrilos,3.37,1.29,154.5,1.80,1.90,0.0,0.00,0.00,zut1lwBE,0.296736,0.775194,0.555556,0.526316,0.071930,214.332,50.759645,0.236827,0.6,1.341641,2.236068,193.12,2.888,0.499570,0.172981,-61.0,105.486,24.196603,0.229382,2.4,1.341641,0.559017,94.43,1.322,0.184986,0.139929,19.0,68,71,2.84,1.33,155.946,134.156,0.631237,0.038222,NaN,-1.92,-0.384,-6.171875,0.000000,0.0,0.000000,3.35,0.670,0.432836,0.000000,0.0,0.000000
2054,2024-06-30,Chile Lnb 2,19:00,Quilpue Basquetbol,Stadio Italiano,3.20,1.32,133.5,1.83,1.87,0.0,0.00,0.00,lbIGPsNe,0.312500,0.757576,0.546448,0.534759,0.070076,113.116,51.006140,0.450919,1.2,1.643168,1.369306,197.34,1.670,0.619879,0.371185,-40.0,204.636,216.491230,1.057933,1.2,1.643168,1.369306,97.97,3.114,3.434389,1.102887,19.0,78,97,2.53,1.01,0.000,206.806,0.588213,0.015289,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,-2.34,-0.468,-0.683761,0.000000,0.0,0.000000
205

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
2057,18:00,Colômbia Lbp,Piratas de Bogota,Toros del Valle,1.77,Back Home
2076,21:05,Colômbia Lbp,Corsarios Cartagena,Motilones del Norte,2.68,Back Home
